<a href="https://colab.research.google.com/github/shiva-tech-code/Defect-Detection-in-wall/blob/main/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install tensorflow opencv-python albumentations pandas matplotlib pillow



In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import albumentations as A
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Mount Google Drive
drive.mount('/content/drive')


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Mounted at /content/drive


In [ ]:

# Define paths
DATA_DIR = '/content/drive/My Drive/dp/Wall_Defects_Dataset'
OUTPUT_DIR = '/content/drive/My Drive/Augmented_Dataset'
CATEGORIES = ['cracks', 'chipping', 'stains', 'paint_flaking', 'holes', 'no_defect']

# Ensure output directory structure
os.makedirs(OUTPUT_DIR, exist_ok=True)
for category in CATEGORIES:
    os.makedirs(os.path.join(OUTPUT_DIR, category), exist_ok=True)

# Simplified Data Augmentation and Preprocessing Pipeline
augmentation_pipeline = A.Compose([
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=10, p=0.3),
    A.HorizontalFlip(p=0.5),
])


In [ ]:

def preprocess_image(image):
    """Preprocess image by converting to grayscale, enhancing contrast, and resizing to 128x128."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_img = clahe.apply(gray)
    resized_img = cv2.resize(enhanced_img, (128, 128))
    return resized_img

# Process each category folder and apply augmentation
for category in CATEGORIES:
    print(f"Processing {category} images...")
    img_dir = os.path.join(DATA_DIR, category)
    augmented_img_dir = os.path.join(OUTPUT_DIR, category)

    for img_name in tqdm(os.listdir(img_dir)):
        img_path = os.path.join(img_dir, img_name)
        try:
            image = cv2.imread(img_path)
            if image is None:
                raise ValueError(f"Failed to load {img_name}")
        except Exception as e:
            print(f"Skipping {img_name} due to error: {e}")
            continue  # Skip corrupted files or unsupported formats

        # Apply preprocessing
        processed_img = preprocess_image(image)

        # Save original processed image
        output_path = os.path.join(augmented_img_dir, f'proc_{img_name}')
        cv2.imwrite(output_path, processed_img)

        # Generate augmented images
        for i in range(2):  # Generate 2 augmented images per input
            augmented = augmentation_pipeline(image=image)['image']
            aug_img_path = os.path.join(augmented_img_dir, f'aug_{i}_{img_name}')
            cv2.imwrite(aug_img_path, augmented)


Processing cracks images...


100%|██████████| 45/45 [01:55<00:00,  2.57s/it]


Processing chipping images...


100%|██████████| 21/21 [00:33<00:00,  1.57s/it]


Processing stains images...


...

100%|██████████| 98/98 [02:31<00:00,  1.54s/it]


Processing holes images...


100%|██████████| 50/50 [01:09<00:00,  1.40s/it]


Processing no_defect images...


100%|██████████| 50/50 [01:08<00:00,  1.37s/it]


In [ ]:

# Load the augmented dataset from Google Drive
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

train_generator = train_datagen.flow_from_directory(
    directory=OUTPUT_DIR,
    target_size=(128, 128),  # Use 128x128 for faster processing
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=OUTPUT_DIR,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)


Found 1003 images belonging to 6 classes.
Found 249 images belonging to 6 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:146: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


In [ ]:

# Build a lightweight model with MobileNetV2
def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False  # Freeze MobileNetV2 layers for feature extraction

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(len(CATEGORIES), activation='softmax')
    ])
    return model


In [ ]:

# Compile and train the model
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model with a reduced number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25 # Reduced epochs for faster training
)

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")



Epoch 1/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 27s 338ms/step - accuracy: 0.2195 - loss: 2.0656 - val_accuracy: 0.5083 - val_loss: 1.3024
Epoch 2/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6250 - loss: 1.3788 - val_accuracy: 0.2222 - val_loss: 1.7204
Epoch 3/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 39s 316ms/step - accuracy: 0.4984 - loss: 1.2795 - val_accuracy: 0.7000 - val_loss: 0.9741
Epoch 4/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6250 - loss: 1.4150 - val_accuracy: 0.6667 - val_loss: 0.9923
Epoch 5/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 21s 321ms/step - accuracy: 0.6706 - loss: 0.9541 - val_accuracy: 0.7750 - val_loss: 0.7943
Epoch 6/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5625 - loss: 0.9863 - val_accuracy: 0.8889 - val_loss: 0.5096
Epoch 7/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 42s 342ms/step - accuracy: 0.7363 - loss: 0.7781 - val_accuracy: 0.7958 - val_loss: 0.6605
Epoch 8/25
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.7005 - val_accuracy: 0.777

In [ ]:
# Save the model
model_save_path = '/content/drive/My Drive/defect_detection_model.h5'
model.save(model_save_path)
print(f"Model saved at {model_save_path}")


Model saved at /content/drive/My Drive/defect_detection_model.h5


In [ ]:

# Function to detect defects in a new image
def detect_defect(img_path, model, categories=CATEGORIES):
    """
    Load a test image, preprocess it, and use the model to predict the defect type.
    Display the image with its predicted label in the terminal.
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict defect type
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_label = categories[predicted_class[0]]

    # Display the image and prediction
    plt.imshow(image.load_img(img_path))
    plt.title(f"Predicted Defect: {predicted_label}")
    plt.axis('off')
    plt.show()

    print(f"Predicted Defect: {predicted_label}")
    return predicted_label


In [ ]:

# Test the function on a new image
test_image_path = '/content/drive/My Drive/dp/testdata.jpg'
detect_defect(test_image_path, model)
